In [1]:
!pip install opencv-python

In [1]:
!pip install dlib
# 설치시 이슈 해결
# 참고 블로그 : https://couchcoding.tistory.com/235

  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dlib: filename=dlib-19.24.6-cp311-cp311-win_amd64.whl size=2885930 sha256=8cf66bd74730bc5e2d74cb761ad0c1b451954e0f4ca713911cd8fc01b75f7420
  Stored in directory: c:\users\ssafy\appdata\local\pip\cache\wheels\fe\c7\1f\c778b9f7cc6d8d0da4f6697f619f9eb5a49d54d2a2c8267f3c
Successfully built dlib


In [4]:
import cv2
import dlib
import random
import time
import numpy as np
import os

# 모델 파일 다운로드 경로와 URL 설정
MODEL_URL = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
MODEL_PATH = "shape_predictor_68_face_landmarks.dat"
FOOD_ICON_PATH = "apple.png"  # apple.png 파일을 음식 아이콘으로 사용

# 모델 파일이 존재하지 않으면 다운로드
def download_file(url, path):
    if not os.path.exists(path):
        print(f"Downloading {path}...")
        urllib.request.urlretrieve(url, path)
        print(f"{path} download complete.")

# 모델 다운로드 실행
download_file(MODEL_URL, MODEL_PATH + ".bz2")

# 모델 압축 해제
if not os.path.exists(MODEL_PATH):
    import bz2
    with bz2.BZ2File(MODEL_PATH + ".bz2") as file, open(MODEL_PATH, 'wb') as uncompressed_file:
        uncompressed_file.write(file.read())
    os.remove(MODEL_PATH + ".bz2")

# dlib의 얼굴 탐지기 및 랜드마크 예측기 사용
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(MODEL_PATH)

# 카메라 연결 (0은 기본 카메라, 노트북의 내장 카메라 사용)
cap = cv2.VideoCapture(0)

# apple.png 파일을 음식 아이콘으로 로드 및 크기 조정
food_icon = cv2.imread(FOOD_ICON_PATH, cv2.IMREAD_UNCHANGED)
food_icon = cv2.resize(food_icon, (50, 50), interpolation=cv2.INTER_AREA)

# 화면에 보여줄 텍스트를 저장할 변수
message = "Press 'q' to quit anytime"
game_instructions = "1: Random Punishment\n2: Follow Expression\n4: Eating Game"

# 게임 상태를 저장하는 변수들
faces_detected = []  # 인식된 얼굴 위치를 저장
game_mode = 0  # 1: 랜덤 벌칙, 2: 표정 따라하기, 4: 음식 먹기 게임
current_task = None  # 현재의 표정이나 방향 등 게임 요구사항
score = 0  # 점수
records = []  # 게임 기록을 저장할 리스트
game_started = False  # 게임이 시작되었는지 여부
show_start_text = False  # 게임 시작 시 START 텍스트 표시
start_time = 0  # 게임 시작 시간
game_over = False  # 게임 종료 여부
selected_face = None  # 1번 게임에서 선택된 얼굴
animation_time = 2  # 애니메이션 시간 (초)
show_confirm_button = False  # 확인 버튼을 표시할지 여부

# 표정 따라하기 게임을 위한 표정 리스트 (임의로 설정)
expressions = ["Smile", "Frown", "Surprised"]

# 음식 먹기 게임 설정
food_positions = []  # 음식들의 위치 리스트
food_speed = 5  # 음식의 이동 속도
max_food_count = 5  # 화면에 동시에 내려오는 음식 개수
game_duration = 15  # 게임 제한 시간 (초)

# 화면 사이즈를 카메라 비율에 맞추도록 설정
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * 1.5)  # 화면 크기를 확대하여 보기 좋게 조정
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * 1.5)

# 확인 버튼 위치 설정
button_position = (frame_width // 2 - 75, frame_height // 2 + 50, 150, 50)  # x, y, width, height

# 텍스트 스타일 설정 함수
def draw_text(img, text, position, font_scale=1, color=(255, 255, 255), thickness=2, font=cv2.FONT_HERSHEY_SIMPLEX, background_color=None):
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    position = (max(10, min(position[0], frame_width - text_size[0] - 10)), 
                max(30, min(position[1], frame_height - 10)))  # 텍스트가 화면을 넘지 않도록 위치 조정
    
    # 배경 색상 설정
    if background_color:
        background_topleft = (position[0] - 10, position[1] - text_size[1] - 10)
        background_bottomright = (position[0] + text_size[0] + 10, position[1] + 10)
        cv2.rectangle(img, background_topleft, background_bottomright, background_color, -1)
    
    cv2.putText(img, text, position, font, font_scale, color, thickness, lineType=cv2.LINE_AA)

# 음식 위치 초기화 함수
def init_food_positions():
    global food_positions
    food_positions = [[random.randint(50, frame_width - 50), 0] for _ in range(max_food_count)]

init_food_positions()

# 마우스 클릭 이벤트 처리 함수
def mouse_callback(event, x, y, flags, param):
    global game_mode, game_started, game_over, selected_face, show_confirm_button
    if event == cv2.EVENT_LBUTTONDOWN:
        # 확인 버튼 클릭 시 홈으로 돌아가기
        if show_confirm_button and button_position[0] <= x <= button_position[0] + button_position[2] and button_position[1] <= y <= button_position[1] + button_position[3]:
            game_mode = 0
            game_over = False
            game_started = False
            selected_face = None
            show_confirm_button = False

# 마우스 콜백 설정
cv2.namedWindow("Game Window")
cv2.setMouseCallback("Game Window", mouse_callback)

while True:
    ret, frame = cap.read()  # 카메라에서 프레임 읽기
    if not ret:
        break
    
    # 카메라 비율에 맞게 프레임을 유지
    frame = cv2.resize(frame, (frame_width, frame_height))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 프레임을 그레이스케일로 변환

    # 얼굴 탐지
    faces = detector(gray)

    # 게임 종료 후 화면 유지
    if game_over and show_confirm_button:
        # 게임 종료 상태에서 점수와 확인 버튼 표시
        draw_text(frame, f"Time's Up! Score: {score}", (frame_width // 2 - 150, frame_height // 2 - 50), font_scale=2, color=(255, 255, 255), background_color=(0, 0, 255))
        # 확인 버튼 그리기
        cv2.rectangle(frame, (button_position[0], button_position[1]), 
                      (button_position[0] + button_position[2], button_position[1] + button_position[3]), (0, 128, 0), -1)
        draw_text(frame, "Confirm", (button_position[0] + 20, button_position[1] + 35), font_scale=1, color=(255, 255, 255), thickness=2, background_color=(0, 128, 0))
        cv2.imshow("Game Window", frame)
        key = cv2.waitKey(1) & 0xFF
        continue  # 화면 유지 및 이벤트 대기

    # 게임 시작 전 "START" 텍스트 표시
    if show_start_text:
        draw_text(frame, "READY TO START!", (frame_width // 2 - 150, frame_height // 2), font_scale=2, color=(50, 205, 50), thickness=3)
        if time.time() - start_time > 3:  # 3초 후 게임 시작
            show_start_text = False
            game_started = True
            start_time = time.time()  # 게임 타이머 초기화

    # 1번 게임: 랜덤 벌칙 게임
    if game_mode == 1 and game_started:
        elapsed_time = time.time() - start_time
        if elapsed_time < animation_time:
            # 얼굴 강조 애니메이션 (랜덤으로 얼굴 선택하여 강조)
            random_face = random.choice(faces)
            cv2.rectangle(frame, (random_face.left(), random_face.top()), 
                          (random_face.right(), random_face.bottom()), (0, 255, 255), 2)
            draw_text(frame, "Who will be chosen?", (50, 50), font_scale=1.5, color=(0, 0, 0), background_color=(255, 255, 0))
        else:
            if selected_face is None:
                # 최종 선택된 얼굴 지정
                selected_face = random.choice(faces)
            # 선택된 얼굴 확대 표시
            x, y, w, h = selected_face.left(), selected_face.top(), selected_face.width(), selected_face.height()
            face_zoom = cv2.resize(frame[y:y+h, x:x+w], (frame_width, frame_height), interpolation=cv2.INTER_CUBIC)
            frame = face_zoom
            draw_text(frame, "You are selected!", (50, 50), font_scale=2, color=(0, 0, 255), background_color=(255, 255, 255))
            draw_text(frame, "Press Confirm", (50, 150), font_scale=1.5, color=(255, 255, 255), background_color=(128, 0, 128))
            # 확인 버튼 표시
            cv2.rectangle(frame, (button_position[0], button_position[1]), 
                          (button_position[0] + button_position[2], button_position[1] + button_position[3]), (0, 128, 0), -1)
            draw_text(frame, "Confirm", (button_position[0] + 20, button_position[1] + 35), font_scale=1, color=(255, 255, 255))
            game_over = True
            show_confirm_button = True

    # 4번 게임: 음식 먹기 게임 모드에서의 동작
    if game_mode == 4 and game_started:
        elapsed_time = time.time() - start_time
        remaining_time = game_duration - int(elapsed_time)

        if remaining_time <= 0:
            # 게임 종료 후 결과 출력
            game_started = False
            game_over = True
            show_confirm_button = True
        else:
            # 현재 점수 표시
            draw_text(frame, f"Score: {score}", (10, 80), font_scale=1.2, color=(255, 215, 0), thickness=2)
            draw_text(frame, f"Time Left: {remaining_time}s", (10, 40), font_scale=1.2, color=(255, 69, 0), thickness=2)

            # 음식 내려오는 애니메이션
            for position in food_positions:
                position[1] += food_speed
                if position[1] > frame_height:  # 음식이 화면 밖으로 나가면 다시 위에서 생성
                    position[0] = random.randint(50, frame_width - 50)
                    position[1] = 0
            
            # 음식 아이콘 그리기
            for position in food_positions:
                y1, y2 = max(position[1], 0), min(position[1] + food_icon.shape[0], frame.shape[0])
                x1, x2 = max(position[0], 0), min(position[0] + food_icon.shape[1], frame.shape[1])
                food_resized = food_icon[0:(y2 - y1), 0:(x2 - x1)]
                overlay = frame[y1:y2, x1:x2]

                # Check if overlay and food_resized are valid and matching in size
                if overlay.shape[:2] == food_resized.shape[:2]:
                    if food_resized.shape[2] == 4:  # 알파 채널이 있는 경우
                        mask = food_resized[:, :, 3]  # 알파 채널을 마스크로 사용
                        mask_inv = cv2.bitwise_not(mask)

                        # 배경과 음식 아이콘의 마스크 처리
                        bg = cv2.bitwise_and(overlay, overlay, mask=mask_inv) if overlay is not None else None
                        fg = cv2.bitwise_and(food_resized[:, :, :3], food_resized[:, :, :3], mask=mask) if food_resized is not None else None

                        if bg is not None and fg is not None:
                            try:
                                frame[y1:y2, x1:x2] = cv2.add(bg, fg)  # 배열 크기와 채널 맞추기
                            except cv2.error as e:
                                print(f"Error combining images: {e}")  # 오류 발생 시 출력

            # 먹는 모션 감지 및 점수 추가
            for face in faces:
                x, y, w, h = face.left(), face.top(), face.width(), face.height()
                shape = predictor(gray, face)
                landmarks = np.array([(shape.part(n).x, shape.part(n).y) for n in range(68)])
                mouth_open = landmarks[66][1] - landmarks[62][1] > 15  # 입이 벌어진 정도 (기준값 조정 가능)

                # 음식이 입 근처에 있을 때 먹는 모션 확인
                for position in food_positions:
                    if position[1] > y and position[1] < y + h and position[0] > x and position[0] < x + w:
                        if mouth_open:
                            score += 10
                            position[0] = random.randint(50, frame_width - 50)
                            position[1] = 0  # 먹은 음식 위치 초기화
                            draw_text(frame, "Yum!", (50, 150), font_scale=2, color=(50, 205, 50), thickness=3)

    # 안내 문구 표시 (게임 시작 전)
    if not game_started and not game_over:
        draw_text(frame, message, (10, 60), font_scale=1.2, color=(255, 255, 255))
        draw_text(frame, game_instructions, (10, 120), font_scale=1, color=(200, 200, 200))

    # 게임 화면 출력
    cv2.imshow("Game Window", frame)

    # 키보드 입력 처리
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key == ord('1'):
        game_mode = 1
        current_task = None
        show_start_text = True
        start_time = time.time()
    elif key == ord('2'):
        game_mode = 2
        current_task = None
        show_start_text = True
        start_time = time.time()
    elif key == ord('4'):
        game_mode = 4
        current_task = None
        show_start_text = True
        start_time = time.time()
        score = 0  # 점수 초기화
        init_food_positions()  # 음식 위치 초기화

cap.release()
cv2.destroyAllWindows()

# 게임이 끝난 후 결과 출력
print("\nGame Results:")
for game, record in records:
    print(f"{game}: {record} points")



Game Results:
